In [70]:
import arcpy

# 取得目前開啟的 ArcGIS Pro 專案
aprx = arcpy.mp.ArcGISProject("CURRENT")
gdb = aprx.defaultGeodatabase 

aprx.defaultGeodatabase = gdb
arcpy.env.workspace = gdb

print("專案路徑：", aprx.filePath)              # .aprx 檔案完整路徑
print("預設 gdb：", aprx.defaultGeodatabase)   # 預設的 geodatabase
print("專案資料夾：", aprx.homeFolder)         # 專案所在的資料夾 (home folder)

# 取得目前活動的 Map (通常就是你 ArcGIS Pro 視窗左邊的那個)
m = aprx.activeMap

print("目前 Map 名稱:", m.name)

sr = m.spatialReference
print("圖層空間參考:", sr.name, sr.factoryCode)

專案路徑： D:\ArcGIS\Morpho_Spatial_Demography\Morpho_Spatial_Demography.aprx
預設 gdb： D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb
專案資料夾： D:\ArcGIS\Morpho_Spatial_Demography
目前 Map 名稱: Hawaii
圖層空間參考: WGS_1984_UTM_Zone_4N 32604


In [38]:
###輸出plot中的小網格

import arcpy
import math
import os

arcpy.env.overwriteOutput = True  # 同名覆蓋
arcpy.env.addOutputsToMap = True  # 自動加到地圖

# ====== 參數 ======
in_fc    = "HW_k1a_plot_8m"   # 輸入 Polygon (圖層或要素類別)
side_len = 0.02                            # 每格邊長 (公尺)
out_fc   = "k1a_grid_2cm" # 輸出 (可用只寫檔名 + 設定 arcpy.env.workspace)

scratch  = arcpy.env.scratchGDB             # 暫存資料放這

# 暫存路徑
fishnet_fc  = os.path.join(scratch, "fishnet_tmp")
clipped_fc  = os.path.join(scratch, "grid_clip_tmp")

# 刪除舊暫存
for tmp in [fishnet_fc, clipped_fc]:
    if arcpy.Exists(tmp):
        arcpy.management.Delete(tmp)

# 1) 取得範圍
ext = arcpy.Describe(in_fc).extent
origin_coord = f"{ext.XMin} {ext.YMin}"
y_axis_coord = f"{ext.XMin} {ext.YMin + 1}"
opp_corner   = f"{ext.XMax} {ext.YMax}"

# 2) 建立 fishnet
arcpy.management.CreateFishnet(
    out_feature_class=fishnet_fc,
    origin_coord=origin_coord,
    y_axis_coord=y_axis_coord,
    cell_width=side_len,
    cell_height=side_len,
    number_rows=None,
    number_columns=None,
    corner_coord=opp_corner,
    labels="NO_LABELS",
    template=in_fc,
    geometry_type="POLYGON"
)

# 3) Clip 到原 Polygon
arcpy.analysis.Clip(fishnet_fc, in_fc, clipped_fc)

# 4) 複製到最終輸出
arcpy.management.CopyFeatures(clipped_fc, out_fc)
print("完成！輸出：", out_fc)

# 5) 清理暫存
for tmp in [fishnet_fc, clipped_fc]:
    if arcpy.Exists(tmp):
        arcpy.management.Delete(tmp)

# 6) 如果 ArcGIS 已經把暫存圖層加到地圖，從 activeMap 移掉它們
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
for name in ["fishnet_tmp", "grid_clip_tmp"]:
    for lyr in m.listLayers(name):
        m.removeLayer(lyr)


完成！輸出： k1a_grid_2cm


In [39]:
#網格編號

import arcpy
import statistics

fc = r"k1a_grid_2cm"
seq_field = "SeqID"

if seq_field not in [f.name for f in arcpy.ListFields(fc)]:
    arcpy.AddField_management(fc, seq_field, "LONG")

cells = []
with arcpy.da.SearchCursor(fc, ["OID@", "SHAPE@XY"]) as cur:
    for oid, (x, y) in cur:
        cells.append((oid, x, y))

if not cells:
    raise ValueError("未讀取到任何幾何。")

ys = sorted([y for _, _, y in cells])
dy = [abs(ys[i+1] - ys[i]) for i in range(len(ys)-1) if abs(ys[i+1] - ys[i]) > 0]

# ✅ 容錯：如果無法估出row_spacing就給一個估計值
if len(dy) > 0:
    row_spacing = statistics.median(dy)
else:
    # 自動估算網格間距（取y範圍/假設10列）
    row_spacing = (max(ys) - min(ys)) / 10.0

# 給安全容差
tol = max(row_spacing * 0.5, 0.001)   # 保證最小容差非0

# === 依 Y 分群 ===
cells.sort(key=lambda r: r[2], reverse=True)
rows = []
current_row = []
anchor_y = cells[0][2]

for rec in cells:
    oid, x, y = rec
    if abs(y - anchor_y) <= tol:
        current_row.append(rec)
    else:
        rows.append(current_row)
        current_row = [rec]
        anchor_y = y
rows.append(current_row)

print(f"偵測到 {len(rows)} 列，每列約 {len(rows[0]) if rows else 0} 格。")

# === 每列左到右排序並編號 ===
seq = 1
oid_to_seq = {}
for row in rows:
    row_sorted = sorted(row, key=lambda r: r[1])
    for oid, _, _ in row_sorted:
        oid_to_seq[oid] = seq
        seq += 1

with arcpy.da.UpdateCursor(fc, ["OID@", seq_field]) as cur:
    for oid, _ in cur:
        cur.updateRow((oid, oid_to_seq[oid]))

print(f"✅ 已完成 SeqID 編號，共 {seq-1} 個格子。")


偵測到 400 列，每列約 400 格。
✅ 已完成 SeqID 編號，共 160000 個格子。


In [41]:
import arcpy

##grid要先由Calculate Geometry Attribute建立x y 資訊###
#####由Polygon Neighbor產生的table在小網格中產生四方鄰居的SeqID資訊#######

# 指定圖層與鄰近關係 table 的名稱
grid_layer = "k1a_grid_2cm"
neighbors_table = "k1a_grid_2cm_PN"

# 設定容許誤差 (若 x, y 差異小於此值則視為相等)
tol = 1e-6

# （可選）印出圖層欄位，檢查欄位名稱
fields = [f.name for f in arcpy.ListFields(grid_layer)]
print("圖層欄位:", fields)

# 建立一個字典，儲存每個網格的 SeqID 與對應的 (x, y) 座標
grid_dict = {}
with arcpy.da.SearchCursor(grid_layer, ["SeqID", "x", "y"]) as cursor:
    for seqid, x, y in cursor:
        grid_dict[seqid] = (x, y)

# 建立結果字典，準備存放各方向的鄰居 SeqID
neighbors_result = {
    seqid: {"up_side": None, "down_side": None, "left_side": None, "right_side": None} 
    for seqid in grid_dict.keys()
}

# 讀取鄰近關係 table，根據座標差異判斷上下左右鄰居
with arcpy.da.SearchCursor(neighbors_table, ["src_SeqID", "nbr_SeqID"]) as cursor:
    for src_seqid, nbr_seqid in cursor:
        # 確保來源與鄰居的 SeqID 都存在
        if src_seqid in grid_dict and nbr_seqid in grid_dict:
            src_x, src_y = grid_dict[src_seqid]
            nbr_x, nbr_y = grid_dict[nbr_seqid]
            dx = nbr_x - src_x
            dy = nbr_y - src_y

            # 若 y 差異接近 0，表示在同一水平線上，判斷左右
            if abs(dy) < tol:
                if dx > tol:
                    neighbors_result[src_seqid]["right_side"] = nbr_seqid
                elif dx < -tol:
                    neighbors_result[src_seqid]["left_side"] = nbr_seqid
            # 若 x 差異接近 0，表示在同一垂直線上，判斷上下
            elif abs(dx) < tol:
                if dy > tol:
                    neighbors_result[src_seqid]["up_side"] = nbr_seqid
                elif dy < -tol:
                    neighbors_result[src_seqid]["down_side"] = nbr_seqid

# 若圖層中不存在用來存放鄰居資訊的欄位，則先新增欄位 (此處型態設定為 LONG)
fields_to_check = ["up_side", "down_side", "left_side", "right_side"]
existing_fields = [f.name for f in arcpy.ListFields(grid_layer)]
for field in fields_to_check:
    if field not in existing_fields:
        arcpy.AddField_management(grid_layer, field, "LONG")

# 使用 UpdateCursor 將計算好的鄰居資訊更新回圖層
with arcpy.da.UpdateCursor(grid_layer, ["SeqID", "up_side", "down_side", "left_side", "right_side"]) as cursor:
    for row in cursor:
        seqid = row[0]
        row[1] = neighbors_result[seqid]["up_side"]
        row[2] = neighbors_result[seqid]["down_side"]
        row[3] = neighbors_result[seqid]["left_side"]
        row[4] = neighbors_result[seqid]["right_side"]
        cursor.updateRow(row)

print("鄰居資訊已成功更新到圖層中！")


圖層欄位: ['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area', 'SeqID', 'x', 'y']
鄰居資訊已成功更新到圖層中！


In [1]:
#-------------------------------------------------------------------
## 需先Add spatial join 珊瑚polgon圖層 存入Colony_ID和Form
# 比對 Colony_ID：相同 → "Self"；不同 → 回傳鄰居 Form (代碼轉文字)
# 若鄰居無 Colony_ID → "Ncoral"
#-------------------------------------------------------------------
import arcpy

# === 1. 輸入資料 ===
fc_grid = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_grid_4cm_2025"

# === 2. 確保輸出欄位存在 ===
for f in ["up_type", "down_type", "left_type", "right_type"]:
    if f not in [field.name for field in arcpy.ListFields(fc_grid)]:
        arcpy.AddField_management(fc_grid, f, "TEXT", field_length=20)

# === 3. 建立 SeqID -> Colony_ID, Form 對照 ===
colony_dict = {}
form_dict = {}

with arcpy.da.SearchCursor(fc_grid, ["SeqID", "Colony_ID", "Form"]) as sCur:
    for seqid, colony, form in sCur:
        colony_dict[seqid] = colony
        form_dict[seqid] = form

# === 4. Form 代碼對照字典（根據你提供的表） ===
form_code_map = {
    1: "br",
    2: "cb",
    3: "di",
    4: "co",
    5: "ta",
    6: "ma",
    7: "en",
    8: "fr",
    9: "fo",
    10: "bu",
    11: "su"
}

# === 5. 定義比較函式 ===
def compare_colony(src_colony, nbr_colony, nbr_form):
    """
    比對自己與鄰居的 Colony_ID：
    - 相同 → "Self"
    - 不同 → 回傳鄰居 Form（轉文字）
    - 鄰居 Colony_ID 空 → "Ncoral"
    """
    # 沒有鄰居 ID → 非珊瑚
    if nbr_colony in (None, "", " "):
        return "Ncoral"
    
    # 同群體
    if src_colony == nbr_colony:
        return "Self"

    # 不同群體，回傳 Form 名稱
    if nbr_form in (None, "", " "):
        return "Unknown"
    try:
        # 嘗試轉換 Form code 為文字
        nbr_form_code = int(nbr_form)
        return form_code_map.get(nbr_form_code, str(nbr_form))
    except Exception:
        # 若本身已是文字 (例如 "br")
        return str(nbr_form)

# === 6. 更新四個方向 ===
fields = [
    "SeqID", "Colony_ID", "Form",
    "up_side", "down_side", "left_side", "right_side",
    "up_type", "down_type", "left_type", "right_type"
]

with arcpy.da.UpdateCursor(fc_grid, fields) as uCur:
    for row in uCur:
        src_seq = row[0]
        src_colony = row[1]
        src_form = row[2]

        up_seq, down_seq, left_seq, right_seq = row[3], row[4], row[5], row[6]

        # 預設 Unknown
        up_result = down_result = left_result = right_result = "Unknown"

        # 上
        if up_seq in colony_dict:
            up_result = compare_colony(src_colony, colony_dict[up_seq], form_dict.get(up_seq))
        # 下
        if down_seq in colony_dict:
            down_result = compare_colony(src_colony, colony_dict[down_seq], form_dict.get(down_seq))
        # 左
        if left_seq in colony_dict:
            left_result = compare_colony(src_colony, colony_dict[left_seq], form_dict.get(left_seq))
        # 右
        if right_seq in colony_dict:
            right_result = compare_colony(src_colony, colony_dict[right_seq], form_dict.get(right_seq))

        # 寫回
        row[7] = up_result
        row[8] = down_result
        row[9] = left_result
        row[10] = right_result

        uCur.updateRow(row)

print("✅ 完成！已根據 Colony_ID 與 Form 編碼更新 up_type / down_type / left_type / right_type。")


✅ 完成！已根據 Colony_ID 與 Form 編碼更新 up_type / down_type / left_type / right_type。


In [44]:
import arcpy

#############計算網格四方鄰居的平均高度資料#######################
######需要先用Zonal Statistics as table計算Cell_Z_mean##########

# 資料來源：請確認路徑正確，並指向 File GDB 中的 Feature Class
fc_grid = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_grid_4cm_2025"

#----------------------------------------------------------------
# 1. 檢查並建立「相鄰網格高度」欄位 (如果不存在就建立)
#----------------------------------------------------------------
height_fields = ["up_hight", "down_hight", "left_hight", "right_hight"]
existing_fields = [f.name for f in arcpy.ListFields(fc_grid)]
for field in height_fields:
    if field not in existing_fields:
        arcpy.AddField_management(fc_grid, field, "DOUBLE")

#----------------------------------------------------------------
# 2. 建立 SeqID -> Cell_Z_mean 對應字典 (以 int(seqid) 作為鍵)
#----------------------------------------------------------------
depth_dict = {}
with arcpy.da.SearchCursor(fc_grid, ["SeqID", "Cell_Z_mean"]) as sCur:
    for seqid, cell_z in sCur:
        if seqid is not None:
            try:
                depth_dict[int(seqid)] = cell_z
            except Exception:
                # 如果 SeqID 無法轉成 int 就略過該筆
                pass

#----------------------------------------------------------------
# 3. 使用 UpdateCursor 更新各網格的相鄰高度值
#    欄位說明：
#       SeqID       : 本網格識別碼 (數值)
#       up_side, down_side, left_side, right_side : 文字型態的相鄰網格識別碼
#       up_hight, down_hight, left_hight, right_hight : 各方向的高度值
#----------------------------------------------------------------
fields = ["SeqID", "up_side", "down_side", "left_side", "right_side",
          "up_hight", "down_hight", "left_hight", "right_hight"]

def convert_to_int(val):
    """若 val 非空字串，轉換成 int；否則回傳 None。"""
    try:
        if val is None or str(val).strip() == "":
            return None
        return int(val)
    except Exception:
        return None

with arcpy.da.UpdateCursor(fc_grid, fields) as uCur:
    for row in uCur:
        # 取得各方向鄰居的 SeqID（文字格式）
        up_side_text    = row[1]
        down_side_text  = row[2]
        left_side_text  = row[3]
        right_side_text = row[4]
        
        # 將文字值轉換為整數 (如果可能)
        up_side    = convert_to_int(up_side_text)
        down_side  = convert_to_int(down_side_text)
        left_side  = convert_to_int(left_side_text)
        right_side = convert_to_int(right_side_text)
        
        # 從 depth_dict 中取得對應的高度值，若不存在則為 None
        up_height    = depth_dict[up_side]    if (up_side is not None and up_side in depth_dict) else None
        down_height  = depth_dict[down_side]  if (down_side is not None and down_side in depth_dict) else None
        left_height  = depth_dict[left_side]  if (left_side is not None and left_side in depth_dict) else None
        right_height = depth_dict[right_side] if (right_side is not None and right_side in depth_dict) else None
        
        # 更新欄位：依序寫入 up_hight, down_hight, left_hight, right_hight
        row[5] = up_height
        row[6] = down_height
        row[7] = left_height
        row[8] = right_height
        
        uCur.updateRow(row)

print("相鄰網格的高度值已更新到 up_hight, down_hight, left_hight, right_hight 欄位！")

相鄰網格的高度值已更新到 up_hight, down_hight, left_hight, right_hight 欄位！


In [45]:
import arcpy

#############計算網格四方鄰居的平均高度差#######################
###### 需要先用 Zonal Statistics as table 或其他方式計算 Cell_Z_mean ########

# 資料來源：請確認路徑正確
fc_grid = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k1a_grid_2cm"

#----------------------------------------------------------------
# 1. 檢查並建立「相鄰網格高度差」欄位 (如果不存在就建立)
#----------------------------------------------------------------
diff_fields = ["up_h_diff", "down_h_diff", "left_h_diff", "right_h_diff"]
existing_fields = [f.name for f in arcpy.ListFields(fc_grid)]
for field in diff_fields:
    if field not in existing_fields:
        arcpy.AddField_management(fc_grid, field, "DOUBLE")

#----------------------------------------------------------------
# 2. 使用 UpdateCursor 根據本網格高度 (Cell_Z_mean) 與相鄰高度計算高度差
#
# 欄位說明：
#   SeqID       : 本網格識別碼
#   Cell_Z_mean : 本網格高度（取代原先的 z_Mean）
#   up_h_diff, down_h_diff, left_h_diff, right_h_diff : 各方向高度差（將被寫入）
#   up_hight, down_hight, left_hight, right_hight : 已更新的相鄰網格高度值（先前步驟需已填入）
#----------------------------------------------------------------
fields = [
    "SeqID",
    "Cell_Z_mean",
    "up_h_diff",
    "down_h_diff",
    "left_h_diff",
    "right_h_diff",
    "up_hight",
    "down_hight",
    "left_hight",
    "right_hight"
]

with arcpy.da.UpdateCursor(fc_grid, fields) as uCur:
    for row in uCur:
        src_height = row[1]  # 本網格的 Cell_Z_mean

        # 取得已更新的相鄰網格高度值（欄位索引依上方 fields）
        up_height    = row[6]
        down_height  = row[7]
        left_height  = row[8]
        right_height = row[9]

        # 計算高度差 = 鄰居高度 - 本網格高度；若任一值為 None，則結果為 None
        up_diff    = up_height - src_height    if (up_height is not None and src_height is not None) else None
        down_diff  = down_height - src_height  if (down_height is not None and src_height is not None) else None
        left_diff  = left_height - src_height  if (left_height is not None and src_height is not None) else None
        right_diff = right_height - src_height if (right_height is not None and src_height is not None) else None

        # 更新高度差欄位（依序寫入 up_h_diff, down_h_diff, left_h_diff, right_h_diff）
        row[2] = up_diff
        row[3] = down_diff
        row[4] = left_diff
        row[5] = right_diff

        uCur.updateRow(row)

print("高度差已更新到 up_h_diff, down_h_diff, left_h_diff, right_h_diff 欄位！")


高度差已更新到 up_h_diff, down_h_diff, left_h_diff, right_h_diff 欄位！


In [50]:
import arcpy
import math

# 設定工作圖層，請確認圖層名稱或路徑正確
layer = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_grid_2cm"

# 要更新的欄位清單
update_fields = ["Angle_West", "Angle_East", "ArcSunlight"]

# 若欄位已存在就刪除再重新建立，以便覆寫
for fld in update_fields:
    try:
        arcpy.DeleteField_management(layer, fld)
    except Exception:
        # 如果欄位不存在，就忽略錯誤
        pass
    arcpy.AddField_management(layer, fld, "DOUBLE")

# 每個網格的水平距離 (4 公分 = 0.04 m)
grid_size = 0.02        ################要記得改

# 使用 UpdateCursor 更新計算結果
with arcpy.da.UpdateCursor(layer, ["Cell_Z_mean", "left_hight", "right_hight",
                                   "Angle_West", "Angle_East", "ArcSunlight"]) as cursor:
    for row in cursor:
        depth     = row[0]  # 本網格的平均高度 Cell_Z_mean
        left_val  = row[1]  # 左側網格高度
        right_val = row[2]  # 右側網格高度
        
        # 若中心高度或左右任一側為空值，則將結果設為空值並更新
        if depth is None or left_val is None or right_val is None:
            row[3] = None
            row[4] = None
            row[5] = None
            cursor.updateRow(row)
            continue
        
        # 計算左側 (西側) 高度差及夾角
        diff_left = left_val - depth
        angle_west = math.degrees(math.atan(diff_left / grid_size))
        
        # 計算右側 (東側) 高度差及夾角
        diff_right = right_val - depth
        angle_east = math.degrees(math.atan(diff_right / grid_size))
        
        # 分別計算各側「逆角」：90° - 夾角
        inv_west = 90 - angle_west
        inv_east = 90 - angle_east
        
        # 日照弧為兩側逆角之和
        arc_sun = inv_west + inv_east

        # 更新欄位值
        row[3] = angle_west
        row[4] = angle_east
        row[5] = arc_sun
        
        cursor.updateRow(row)

print("夾角與日照弧計算完成（使用 Cell_Z_mean 欄位）！")


夾角與日照弧計算完成（使用 Cell_Z_mean 欄位）！


In [51]:
import pandas as pd
import numpy as np
import arcpy
import os

#==============================================================
# 🧭【參數設定區 — 只需修改這裡】 #coral記得做zonal計算polygon內的DEM mean STD
#==============================================================

fc_grid  = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_grid_2cm"
fc_coral = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\k2b_2308_corals"
out_csv  = r"D:\ArcGIS\Morpho_Spatial_Demography\data\k2b_grid_2cm_summary.csv"

cell_size_m = 0.02                 ##########要記得改
cell_area_m2 = cell_size_m ** 2

coral_forms = ["br","cb","di","co","ta","ma","en","fr","fo","bu","su"]
form_domain = {
    1:"br",2:"cb",3:"di",4:"co",5:"ta",6:"ma",
    7:"en",8:"fr",9:"fo",10:"bu",11:"su"
}

#==============================================================
# 1. 從 GIS 圖層讀取屬性表（用 dict 組 DataFrame）
#==============================================================
if not arcpy.Exists(fc_grid):
    raise FileNotFoundError(f"❌ 找不到圖層：{fc_grid}")

fields_grid = [
    "Colony_ID","Cell_Z_Mean","ArcSunlight",
    "up_type","down_type","left_type","right_type",
    "up_h_diff","down_h_diff","left_h_diff","right_h_diff"
]

# 用 SearchCursor 取代 TableToNumPyArray（不再有 dtype 限制）
records = []
with arcpy.da.SearchCursor(fc_grid, fields_grid) as cursor:
    for row in cursor:
        records.append(list(row))

df = pd.DataFrame(records, columns=fields_grid)

# 空值轉 NaN
df = df.replace({None: np.nan})

#==============================================================
# 2. type 欄位統一轉小寫文字
#==============================================================
for col in ["up_type","down_type","left_type","right_type"]:
    df[col] = df[col].astype(str).str.lower().str.strip()

#==============================================================
# 3. 計算四邊類型次數
#==============================================================
base_types = ["Self","Ncoral","Coral","Unknow"]
for col in base_types + coral_forms:
    df[f"{col}_count"] = 0

type_cols = ["up_type","down_type","left_type","right_type"]

def count_type_detailed(row, cols):
    counts = {k: 0 for k in base_types + coral_forms}
    for c in cols:
        val = row[c]
        if val == "self":
            counts["Self"] += 1
        elif val == "ncoral":
            counts["Ncoral"] += 1
        elif val in coral_forms:
            counts[val] += 1
            counts["Coral"] += 1
        elif val == "coral":
            counts["Coral"] += 1
        else:
            counts["Unknow"] += 1
    return counts

for i, r in df.iterrows():
    c = count_type_detailed(r, type_cols)
    for k, v in c.items():
        df.at[i, f"{k}_count"] = v

#==============================================================
# 4. 建立 Pair 資料表（改用 *_h_diff 欄位）
#==============================================================
rows = []
for _, row in df.iterrows():
    cid = row["Colony_ID"]
    pairs = [
        (row["up_type"], row["up_h_diff"]),
        (row["down_type"], row["down_h_diff"]),
        (row["left_type"], row["left_h_diff"]),
        (row["right_type"], row["right_h_diff"])
    ]
    for ptype, ph in pairs:
        rows.append({"Colony_ID": cid, "PairType": ptype, "PairHight": ph})

df_pairs = pd.DataFrame(rows)
df_pairs["PairHight"] = pd.to_numeric(df_pairs["PairHight"], errors="coerce")

#==============================================================
# 5. 計算 T/NC/C_MarRH（不限制小數位）
#==============================================================
def mean_if_valid(sub, cond):
    x = sub.loc[cond & pd.notnull(sub["PairHight"]), "PairHight"]
    return x.mean() if len(x) else np.nan  # ⬅️ 移除 round()

def get_T(sub):  return mean_if_valid(sub, sub["PairType"] != "self")
def get_NC(sub): return mean_if_valid(sub, sub["PairType"] == "ncoral")
def get_C(sub):  return mean_if_valid(sub, sub["PairType"].isin(["coral"] + coral_forms))

df_marRH = (
    df_pairs.groupby("Colony_ID")
    .apply(lambda g: pd.Series({
        "T_MarRH": get_T(g),
        "NC_MarRH": get_NC(g),
        "C_MarRH": get_C(g)
    }))
    .reset_index()
)

#==============================================================
# 6. 匯總次數 + 計算格網面積
#==============================================================
group_cols = ["Self_count","Ncoral_count","Coral_count","Unknow_count"] + [f"{f}_count" for f in coral_forms]
gdf = df.groupby("Colony_ID", as_index=False)[group_cols].sum()

gdf["Total_cells"] = (
    gdf["Self_count"] + gdf["Ncoral_count"] + gdf["Coral_count"] + gdf["Unknow_count"]
) / 4.0
gdf["Raster_Area"] = gdf["Total_cells"] * cell_area_m2
gdf["Raster_Perimeter"] = (
    gdf["Ncoral_count"] + gdf["Coral_count"] + gdf["Unknow_count"]
) * cell_size_m

#==============================================================
# 7. Edge 與 Colony 層級 Cell_Z_Mean / ArcSunlight 統計
#==============================================================
def is_edge(row):
    return any(row[c] != "self" for c in type_cols if pd.notna(row[c]))

df["is_edge"] = df.apply(is_edge, axis=1)

edge_stats = (
    df[df["is_edge"]]
    .groupby("Colony_ID")
    .agg(
        Edge_CellZ_mean=("Cell_Z_Mean","mean"),
        Edge_CellZ_std=("Cell_Z_Mean","std"),
        Edge_Sun_mean=("ArcSunlight","mean"),
        Edge_Sun_std=("ArcSunlight","std")
    )
    .reset_index()
)

colony_stats = (
    df.groupby("Colony_ID")
    .agg(
        Colony_CellZ_mean=("Cell_Z_Mean","mean"),
        Colony_CellZ_std=("Cell_Z_Mean","std"),
        Colony_Sun_mean=("ArcSunlight","mean"),
        Colony_Sun_std=("ArcSunlight","std")
    )
    .reset_index()
)

for d in [edge_stats, colony_stats]:
    for c in d.columns:
        if c != "Colony_ID":
            d[c] = d[c].round(3)

gdf = gdf.merge(edge_stats, on="Colony_ID", how="left")
gdf = gdf.merge(colony_stats, on="Colony_ID", how="left")

#==============================================================
# 8. 合併格網統計與高度平均
#==============================================================
merge_df = pd.merge(gdf, df_marRH, on="Colony_ID", how="left")

#==============================================================
# 9. 從 coral layer 匯入真實資料
#==============================================================
arr = arcpy.da.TableToNumPyArray(
    fc_coral,
    ["ID","Form","Shape_Area","Shape_Length","Colony_Z_Mean","Colony_Z_STD"],
    skip_nulls=False,
    null_value=np.nan
)
df_coral = pd.DataFrame(arr)

def clean_id(v):
    if pd.isna(v): return ""
    s = str(v).strip()
    if s.endswith(".0"): s = s[:-2]
    return s

df_coral["Colony_ID"] = df_coral["ID"].apply(clean_id)
merge_df["Colony_ID"] = merge_df["Colony_ID"].apply(clean_id)

def translate_form(v):
    try:
        code = int(v)
        return form_domain.get(code, str(v))
    except:
        return str(v).strip().lower() if pd.notna(v) else ""

df_coral["Form"] = df_coral["Form"].apply(translate_form)

merge_df = merge_df.merge(
    df_coral[["Colony_ID","Form","Shape_Area","Shape_Length","Colony_Z_Mean","Colony_Z_STD"]],
    on="Colony_ID", how="left"
)

#==============================================================
# 10. 輸出結果
#==============================================================
merge_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print("✅ 已完成：", os.path.basename(out_csv))
print(f"📊 共 {len(merge_df)} 筆資料")
print("主要欄位：", list(merge_df.columns[:15]))


C:/Users/keelu/AppData/Local/Temp/ArcGISProTemp23600/xpython_23600/3781349290.py:114: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


✅ 已完成： k2b_grid_2cm_summary.csv
📊 共 993 筆資料
主要欄位： ['Colony_ID', 'Self_count', 'Ncoral_count', 'Coral_count', 'Unknow_count', 'br_count', 'cb_count', 'di_count', 'co_count', 'ta_count', 'ma_count', 'en_count', 'fr_count', 'fo_count', 'bu_count']
